In [133]:
import torch
import matplotlib.pyplot as plt

from tqdm import tqdm
from torchvision import datasets
from torchvision.transforms import ToTensor

In [134]:
train_data = datasets.MNIST(
    root = 'data',
    train = True,
    transform=ToTensor(),
    download=False
)

train_data = datasets.MNIST(
    root = 'data',
    train = True,
    transform=ToTensor(),
    download=False
)

In [135]:
type(train_data)

torchvision.datasets.mnist.MNIST

In [136]:
train_data.data.shape

torch.Size([60000, 28, 28])

In [137]:
test_data.data.shape

torch.Size([10000, 28, 28])

In [138]:
train_data.targets

tensor([5, 0, 4,  ..., 5, 6, 8])

In [139]:
from torch.utils.data import DataLoader

loaders = {
    
    'train': DataLoader(train_data, 
                        batch_size=64, 
                        shuffle=True, 
                        num_workers=1),
    
    'test': DataLoader(test_data, 
                       batch_size=64, 
                       shuffle=True, 
                       num_workers=1)
}

In [140]:
loaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x1a1c090ed90>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x1a1c0880810>}

In [141]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        
        return F.softmax(x)

In [142]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CNN().to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)

loss_fn = nn.CrossEntropyLoss()

def train(epoch):
    model.train()
    for batch_inx, (data, target) in enumerate(loaders['train']):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        if batch_inx % 20 == 0:
            print(f'Train Epoch: {epoch} [{batch_inx * len(data)}/{len(loaders["train"].dataset)} ({100. * batch_inx / len(loaders["train"]):.0f}%)]\t{loss.item():.6f}')
            
def test():
    model.eval()
    
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in loaders['test']:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_fn(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    test_loss /= len(loaders['test'].dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy {correct}/{len(loaders["test"].dataset)} ({100. * correct / len(loaders["test"].dataset):.0f}%)\n')
    
        

In [143]:
for epoch in tqdm(range(1,11)):
    train(epoch)
    test()

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]C:\Users\Stepan\AppData\Local\Temp\ipykernel_12680\3577856844.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(x)


Train Epoch: 1 [0/60000 (0%)]	2.303942
Train Epoch: 1 [1280/60000 (2%)]	2.298735
Train Epoch: 1 [2560/60000 (4%)]	2.233627
Train Epoch: 1 [3840/60000 (6%)]	2.083380
Train Epoch: 1 [5120/60000 (9%)]	1.872555
Train Epoch: 1 [6400/60000 (11%)]	1.942014
Train Epoch: 1 [7680/60000 (13%)]	1.805330
Train Epoch: 1 [8960/60000 (15%)]	1.728158
Train Epoch: 1 [10240/60000 (17%)]	1.770471
Train Epoch: 1 [11520/60000 (19%)]	1.659407
Train Epoch: 1 [12800/60000 (21%)]	1.655198
Train Epoch: 1 [14080/60000 (23%)]	1.701814
Train Epoch: 1 [15360/60000 (26%)]	1.790240
Train Epoch: 1 [16640/60000 (28%)]	1.776806
Train Epoch: 1 [17920/60000 (30%)]	1.676797
Train Epoch: 1 [19200/60000 (32%)]	1.648208
Train Epoch: 1 [20480/60000 (34%)]	1.690937
Train Epoch: 1 [21760/60000 (36%)]	1.733795
Train Epoch: 1 [23040/60000 (38%)]	1.822145
Train Epoch: 1 [24320/60000 (41%)]	1.612004
Train Epoch: 1 [25600/60000 (43%)]	1.615328
Train Epoch: 1 [26880/60000 (45%)]	1.650576
Train Epoch: 1 [28160/60000 (47%)]	1.626910
Trai

 10%|████████▎                                                                          | 1/10 [00:11<01:42, 11.37s/it]


Test set: Average loss: 0.0240, Accuracy 9354/10000 (94%)

Train Epoch: 2 [0/60000 (0%)]	1.672798
Train Epoch: 2 [1280/60000 (2%)]	1.571594
Train Epoch: 2 [2560/60000 (4%)]	1.675455
Train Epoch: 2 [3840/60000 (6%)]	1.641491
Train Epoch: 2 [5120/60000 (9%)]	1.618023
Train Epoch: 2 [6400/60000 (11%)]	1.660507
Train Epoch: 2 [7680/60000 (13%)]	1.632256
Train Epoch: 2 [8960/60000 (15%)]	1.580621
Train Epoch: 2 [10240/60000 (17%)]	1.565180
Train Epoch: 2 [11520/60000 (19%)]	1.602217
Train Epoch: 2 [12800/60000 (21%)]	1.578446
Train Epoch: 2 [14080/60000 (23%)]	1.601257
Train Epoch: 2 [15360/60000 (26%)]	1.606237
Train Epoch: 2 [16640/60000 (28%)]	1.595900
Train Epoch: 2 [17920/60000 (30%)]	1.612998
Train Epoch: 2 [19200/60000 (32%)]	1.621710
Train Epoch: 2 [20480/60000 (34%)]	1.622534
Train Epoch: 2 [21760/60000 (36%)]	1.590150
Train Epoch: 2 [23040/60000 (38%)]	1.632145
Train Epoch: 2 [24320/60000 (41%)]	1.618531
Train Epoch: 2 [25600/60000 (43%)]	1.597469
Train Epoch: 2 [26880/60000 (45%

 20%|████████████████▌                                                                  | 2/10 [00:22<01:31, 11.40s/it]

...


Test set: Average loss: 0.0234, Accuracy 9734/10000 (97%)

Train Epoch: 9 [0/60000 (0%)]	1.564430
Train Epoch: 9 [1280/60000 (2%)]	1.524496
Train Epoch: 9 [2560/60000 (4%)]	1.556342
Train Epoch: 9 [3840/60000 (6%)]	1.517770
Train Epoch: 9 [5120/60000 (9%)]	1.486652
Train Epoch: 9 [6400/60000 (11%)]	1.530646
Train Epoch: 9 [7680/60000 (13%)]	1.517375
Train Epoch: 9 [8960/60000 (15%)]	1.506211
Train Epoch: 9 [10240/60000 (17%)]	1.557037
Train Epoch: 9 [11520/60000 (19%)]	1.493624
Train Epoch: 9 [12800/60000 (21%)]	1.542131
Train Epoch: 9 [14080/60000 (23%)]	1.524093
Train Epoch: 9 [15360/60000 (26%)]	1.508085
Train Epoch: 9 [16640/60000 (28%)]	1.576309
Train Epoch: 9 [17920/60000 (30%)]	1.549892
Train Epoch: 9 [19200/60000 (32%)]	1.495516
Train Epoch: 9 [20480/60000 (34%)]	1.511778
Train Epoch: 9 [21760/60000 (36%)]	1.564602
Train Epoch: 9 [23040/60000 (38%)]	1.484360
Train Epoch: 9 [24320/60000 (41%)]	1.602854
Train Epoch: 9 [25600/60000 (43%)]	1.478404
Train Epoch: 9 [26880/60000 (45%

 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [01:44<00:11, 11.71s/it]


Test set: Average loss: 0.0234, Accuracy 9721/10000 (97%)

Train Epoch: 10 [0/60000 (0%)]	1.544639
Train Epoch: 10 [1280/60000 (2%)]	1.528710
Train Epoch: 10 [2560/60000 (4%)]	1.529637
Train Epoch: 10 [3840/60000 (6%)]	1.516713
Train Epoch: 10 [5120/60000 (9%)]	1.524585
Train Epoch: 10 [6400/60000 (11%)]	1.499704
Train Epoch: 10 [7680/60000 (13%)]	1.526732
Train Epoch: 10 [8960/60000 (15%)]	1.493801
Train Epoch: 10 [10240/60000 (17%)]	1.512568
Train Epoch: 10 [11520/60000 (19%)]	1.489039
Train Epoch: 10 [12800/60000 (21%)]	1.598248
Train Epoch: 10 [14080/60000 (23%)]	1.554094
Train Epoch: 10 [15360/60000 (26%)]	1.529904
Train Epoch: 10 [16640/60000 (28%)]	1.523878
Train Epoch: 10 [17920/60000 (30%)]	1.601758
Train Epoch: 10 [19200/60000 (32%)]	1.526245
Train Epoch: 10 [20480/60000 (34%)]	1.542119
Train Epoch: 10 [21760/60000 (36%)]	1.567714
Train Epoch: 10 [23040/60000 (38%)]	1.511817
Train Epoch: 10 [24320/60000 (41%)]	1.463553
Train Epoch: 10 [25600/60000 (43%)]	1.488443
Train Epoch

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:56<00:00, 11.68s/it]


Test set: Average loss: 0.0234, Accuracy 9731/10000 (97%)



In [144]:
device

device(type='cuda')

In [145]:
torch_tenzor = torch.randn(3,4,5, device=device)
print(torch_tenzor.shape,'\n', torch_tenzor)

torch.Size([3, 4, 5]) 
 tensor([[[-1.3546, -0.6444, -0.8354,  3.0058, -3.0112],
         [-0.3302, -0.3510,  0.4539, -1.2193,  1.7731],
         [ 0.8176,  0.5375,  0.9117, -1.5843, -0.0984],
         [-0.9933,  0.0553,  0.1760,  0.1377, -0.4918]],

        [[-0.0156,  0.1002,  2.0079, -1.2036, -0.3101],
         [ 0.5766, -0.4363, -0.6329, -0.6787, -0.2054],
         [ 0.4182, -0.1660,  1.3155,  1.0740, -0.3726],
         [ 0.1030,  0.3462, -0.0268, -0.8327, -1.4514]],

        [[-1.2844, -0.1269, -1.3423, -0.9462, -0.1918],
         [-0.6804, -0.3643,  1.3263, -1.6483, -0.8329],
         [-0.4506, -1.6685,  0.4311, -0.3694, -0.2759],
         [-0.2191,  0.0126,  0.3336,  0.2443, -0.6055]]], device='cuda:0')


In [146]:
torch_tenzor = torch_tenzor.view(6,10)
torch_tenzor

tensor([[-1.3546, -0.6444, -0.8354,  3.0058, -3.0112, -0.3302, -0.3510,  0.4539,
         -1.2193,  1.7731],
        [ 0.8176,  0.5375,  0.9117, -1.5843, -0.0984, -0.9933,  0.0553,  0.1760,
          0.1377, -0.4918],
        [-0.0156,  0.1002,  2.0079, -1.2036, -0.3101,  0.5766, -0.4363, -0.6329,
         -0.6787, -0.2054],
        [ 0.4182, -0.1660,  1.3155,  1.0740, -0.3726,  0.1030,  0.3462, -0.0268,
         -0.8327, -1.4514],
        [-1.2844, -0.1269, -1.3423, -0.9462, -0.1918, -0.6804, -0.3643,  1.3263,
         -1.6483, -0.8329],
        [-0.4506, -1.6685,  0.4311, -0.3694, -0.2759, -0.2191,  0.0126,  0.3336,
          0.2443, -0.6055]], device='cuda:0')

In [147]:
torch_inttenzor = torch.randint(10, (10,), device=device)
torch_inttenzor

tensor([9, 2, 5, 6, 2, 8, 5, 9, 9, 8], device='cuda:0')

In [148]:
result_tenzor = torch_tenzor * torch_inttenzor
result_tenzor

tensor([[-12.1915,  -1.2888,  -4.1769,  18.0346,  -6.0225,  -2.6418,  -1.7552,
           4.0854, -10.9737,  14.1846],
        [  7.3585,   1.0751,   4.5587,  -9.5057,  -0.1967,  -7.9462,   0.2763,
           1.5844,   1.2395,  -3.9340],
        [ -0.1403,   0.2005,  10.0395,  -7.2215,  -0.6203,   4.6132,  -2.1815,
          -5.6963,  -6.1081,  -1.6430],
        [  3.7642,  -0.3320,   6.5774,   6.4437,  -0.7453,   0.8236,   1.7311,
          -0.2410,  -7.4941, -11.6112],
        [-11.5598,  -0.2538,  -6.7114,  -5.6770,  -0.3835,  -5.4433,  -1.8216,
          11.9371, -14.8348,  -6.6634],
        [ -4.0556,  -3.3371,   2.1553,  -2.2164,  -0.5518,  -1.7530,   0.0630,
           3.0023,   2.1990,  -4.8438]], device='cuda:0')

In [149]:
result_tenzor.shape
result_tenzor_T = result_tenzor.t()
print(result_tenzor.shape, result_tenzor_T.shape)

torch.Size([6, 10]) torch.Size([10, 6])


In [150]:
result = torch.matmul(reshaped_tensor.to(device=device), result_tenzor_T.to(device))
result.shape

torch.Size([6, 6])

In [151]:
x = torch.ones(3, requires_grad=True)
z = torch.ones(3, requires_grad=True)
t = torch.ones(3, requires_grad=True)

y = x**3 + z - 75*t #(1, 0.5, 2)
y.backward(torch.ones_like(y))

grad_x = x.grad
grad_z = z.grad
grad_t = t.grad


print("Градиенты:")
print("По x:", grad_x)
print("По z:", grad_z)
print("По t:", grad_t)

Градиенты:
По x: tensor([3., 3., 3.])
По z: tensor([1., 1., 1.])
По t: tensor([-75., -75., -75.])


In [152]:
onec_tenzor_to_numpy = torch.ones(5,6).numpy()
onec_tenzor_to_numpy

array([[1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1.]], dtype=float32)